# **Creating RDF triples**

RDF allows us to make statements about resources. A statement always has the following structure:\
`<subject> <predicate> <object>`.

An RDF statement expresses a relationship between two resources. The subject and the object represent the two resources being related; the predicate represents the nature of their relationship. The relationship is phrased in a directional way (from subject to object) and is called in RDF a property. Because RDF statements consist of three elements they are called triples.

In [ ]:
!pip install rdflib

## Creating Nodes

The subjects and objects of the triples make up the nodes in the graph where the nodes are URI references, Blank Nodes or Literals. In RDFLib, these node types are represented by the classes **URIRef**, **BNode**, **Literal**. *URIRefs* and *Bnodes* can both be thought of as resources, such a person, a company, a website, etc.
- A *BNode* is a node where the exact URI is not known.
- A *URIRef* is a node where the exact URI is know. *URIRefs* are also used to represent the properties/predicates in the RDF graph.
- *Literals* represent attribute values, such as a name, a date, a number, etc. The most common literal values are XML data types, e.g. string, int..

## Example RDF Graph

![RDF Graph](https://www.w3.org/TR/rdf11-primer/example-graph-iris.jpg)

### (Informal) Representation of the Graph
`<Bob> <is a> <person>.`\
`<Bob> <is a friend of> <Alice>.`\
`<Bob> <is born on> <the 4th of July 1990>.`\
`<Bob> <is interested in> <the Mona Lisa>.`\
`<the Mona Lisa> <was created by> <Leonardo da Vinci>.`\
`<the video 'La Joconde à Washington'> <is about> <the Mona Lisa>`

In [2]:
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO

mona_lisa = URIRef('http://www.wikidata.org/entity/Q12418')
davinci = URIRef('http://dbpedia.org/resource/Leonardo_da_Vinci')
lajoconde = URIRef('http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619')

EX = Namespace('http://example.org/')
bob = EX['Bob']
alice = EX['Alice']

birth_date = Literal("1990-07-04", datatype=XSD['date'])
title = Literal('Mona Lisa', lang='en')

In [8]:
title.value

'Mona Lisa'

In [4]:
from rdflib import Graph
g = Graph()

# Bind prefix to namespace
g.bind('ex', EX)
g.bind('foaf', FOAF)
g.bind('schema', SDO)
g.bind('dcterms', DCTERMS)

g.add((bob, RDF.type, FOAF.Person))
g.add((bob, FOAF.knows, alice))
g.add((bob, FOAF['topic_interest'], mona_lisa))
g.add((bob, SDO['birthDate'], birth_date))

g.add((mona_lisa, DCTERMS['creator'], davinci))
g.add((mona_lisa, DCTERMS['title'], title))
g.add((lajoconde, DCTERMS['subject'], mona_lisa))

In [10]:
print(g.serialize(format='xml').decode('u8'))

<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF
   xmlns:dcterms="http://purl.org/dc/terms/"
   xmlns:foaf="http://xmlns.com/foaf/0.1/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:schema="https://schema.org/"
>
  <rdf:Description rdf:about="http://example.org/Bob">
    <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
    <foaf:topic_interest rdf:resource="http://www.wikidata.org/entity/Q12418"/>
    <schema:birthDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">1990-07-04</schema:birthDate>
    <foaf:knows rdf:resource="http://example.org/Alice"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.wikidata.org/entity/Q12418">
    <dcterms:creator rdf:resource="http://dbpedia.org/resource/Leonardo_da_Vinci"/>
    <dcterms:title xml:lang="en">Mona Lisa</dcterms:title>
  </rdf:Description>
  <rdf:Description rdf:about="http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619">
    <dcterms:subject rdf:resource="http

In [ ]:
# Bind prefix to namespace
# g.bind('ex', EX)
# g.bind('foaf', FOAF)
# g.bind('schema', SDO)
# g.bind('dcterms', DCTERMS)

In [11]:
for prefix, ns in g.namespaces():
    print(prefix, ns)

xml http://www.w3.org/XML/1998/namespace
rdf http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs http://www.w3.org/2000/01/rdf-schema#
xsd http://www.w3.org/2001/XMLSchema#
ex http://example.org/
foaf http://xmlns.com/foaf/0.1/
schema https://schema.org/
dcterms http://purl.org/dc/terms/


In [13]:
# Replace Literal value
g.set((bob, SDO['birthDate'], Literal('1990-01-01', datatype=XSD.date)))
g.set((mona_lisa, DCTERMS['title'], Literal('La Joconde', lang='fr')))

In [14]:
print(g.serialize(format='ttl').decode('u8'))

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619> dcterms:subject <http://www.wikidata.org/entity/Q12418> .

ex:Bob a foaf:Person ;
    foaf:knows ex:Alice ;
    foaf:topic_interest <http://www.wikidata.org/entity/Q12418> ;
    schema:birthDate "1990-01-01"^^xsd:date .

<http://www.wikidata.org/entity/Q12418> dcterms:creator <http://dbpedia.org/resource/Leonardo_da_Vinci> ;
    dcterms:title "La Joconde"@fr .




In [15]:
# Remove triples from graph
g.remove((mona_lisa, None, None))

In [16]:
print(g.serialize(format='ttl').decode('utf-8'))

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619> dcterms:subject <http://www.wikidata.org/entity/Q12418> .

ex:Bob a foaf:Person ;
    foaf:knows ex:Alice ;
    foaf:topic_interest <http://www.wikidata.org/entity/Q12418> ;
    schema:birthDate "1990-01-01"^^xsd:date .


